In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd

from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, IntegerType, DateType, StringType, DoubleType
from pyspark.sql.functions import udf, isnan, when, count, col, coalesce, lit, avg, mean, concat, concat_ws

In [2]:
import os 
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/majo/Documents/david/mariadb-java-client-2.5.3.jar pyspark-shell'

In [3]:
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession
spark.sparkContext.setLogLevel("WARN")

21/11/16 00:36:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/16 00:36:33 WARN DependencyUtils: Local jar /Users/majo/Documents/david/mariadb-java-client-2.5.3.jar does not exist, skipping.
21/11/16 00:36:34 INFO SparkContext: Running Spark version 3.2.0
21/11/16 00:36:34 INFO ResourceUtils: ==============================================================
21/11/16 00:36:34 INFO ResourceUtils: No custom resources configured for spark.driver.
21/11/16 00:36:34 INFO ResourceUtils: ==============================================================
21/11/16 00:36:34 INFO SparkContext: Submitted application: pyspark-shell
21/11/16 00:36:34 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap

In [4]:
#PATH = "/Users/majo/Desktop/Maestria_MIAD/Semestre_1/Modelado_de_Datos_y_ETL/Semana_2/Tarea_Data_Profiling/"

# Perfilamiento y Transformacion de los Datos

# Aeropuertos
 Realizaremos un análisis exploratorio del conjunto de datos del archivo vuelos y obtendremos algunas observaciones.

In [5]:
aeropuertos = spark.read.load("aeropuertos.csv", format = "csv", sep = ",", inferSchema = "true", header = "true")

In [6]:
print(aeropuertos.schema)

StructType(List(StructField(_c0,IntegerType,true),StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,DoubleType,true),StructField(longitud,DoubleType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,DoubleType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(elevacion,DoubleType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,StringType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))


### Tipos de las variables del dataframe

In [7]:
aeropuertos.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- sigla: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- propietario: string (nullable = true)
 |-- explotador: string (nullable = true)
 |-- longitud_pista: double (nullable = true)
 |-- ancho_pista: double (nullable = true)
 |-- pbmo: string (nullable = true)
 |-- elevacion: double (nullable = true)
 |-- resolucion: string (nullable = true)
 |-- fecha_construccion: string (nullable = true)
 |-- fecha_vigencia: string (nullable = true)
 |-- clase: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- numero_vuelos_origen: string (nullable = true)
 |-- gcd_departamento: integer (nullable = true)
 |-- gcd_municipio: integer (nullable = true)



### Forma de los datos

In [8]:
print(f'row_number: {aeropuertos.count()} ,', f'columns_number: {len(aeropuertos.columns)}')

row_number: 292 , columns_number: 23


### Selección de datos

De acuerdo al modelo, se seleccionara las variables o columnas a tener en cuenta


<img src="modelo.png">

In [9]:
aeropuertos = aeropuertos.selectExpr('sigla','fecha_construccion as idFechaConstruccion',
                                  'nombre as nombreAeropuerto', 'categoria','tipo', 'departamento','municipio',
                                  'latitud','longitud','propietario','longitud_pista as longitudPista',
                                  'ancho_pista as anchoPista', 'pbmo','elevacion')

## Filtrar datos fechas
Las fechas de construcción no pueden ser mayores a 2013

In [10]:
aeropuertos = aeropuertos.where((aeropuertos['idFechaConstruccion']<'2014-01-01'))

In [11]:
print(f'row_number: {aeropuertos.count()} ,', f'columns_number: {len(aeropuertos.columns)}')

row_number: 242 , columns_number: 14


### Valores Nulos

In [12]:
aeropuertos.select([count(when(isnan(c), c)).alias(c) for c in aeropuertos.columns]).show()

+-----+-------------------+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+
|sigla|idFechaConstruccion|nombreAeropuerto|categoria|tipo|departamento|municipio|latitud|longitud|propietario|longitudPista|anchoPista|pbmo|elevacion|
+-----+-------------------+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+
|    0|                  0|               0|        0|   0|           0|        0|      0|       0|          3|            0|         0|  54|        0|
+-----+-------------------+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+



In [13]:
#aeropuertos.where(col("idFechaConstruccion").isNull()).show()

### Cast Column Type

In [14]:
aeropuertos.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- idFechaConstruccion: string (nullable = true)
 |-- nombreAeropuerto: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- propietario: string (nullable = true)
 |-- longitudPista: double (nullable = true)
 |-- anchoPista: double (nullable = true)
 |-- pbmo: string (nullable = true)
 |-- elevacion: double (nullable = true)



In [15]:
aeropuertos = aeropuertos.withColumn("idFechaConstruccion",aeropuertos.idFechaConstruccion.cast(DateType()))
aeropuertos = aeropuertos.withColumn("nombreAeropuerto",aeropuertos.nombreAeropuerto.cast(StringType()))
aeropuertos = aeropuertos.withColumn("categoria",aeropuertos.categoria.cast(StringType()))
aeropuertos = aeropuertos.withColumn("tipo",aeropuertos.tipo.cast(StringType()))
aeropuertos = aeropuertos.withColumn("departamento",aeropuertos.departamento.cast(StringType()))
aeropuertos = aeropuertos.withColumn("municipio",aeropuertos.municipio.cast(StringType()))
aeropuertos = aeropuertos.withColumn("latitud",aeropuertos.latitud.cast(FloatType()))
aeropuertos = aeropuertos.withColumn("longitud",aeropuertos.longitud.cast(FloatType()))
aeropuertos = aeropuertos.withColumn("propietario",aeropuertos.propietario.cast(StringType()))
aeropuertos = aeropuertos.withColumn("longitudPista",aeropuertos.longitudPista.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("anchoPista",aeropuertos.anchoPista.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("pbmo",aeropuertos.pbmo.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("elevacion",aeropuertos.elevacion.cast(IntegerType()))

In [16]:
aeropuertos.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- idFechaConstruccion: date (nullable = true)
 |-- nombreAeropuerto: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: float (nullable = true)
 |-- longitud: float (nullable = true)
 |-- propietario: string (nullable = true)
 |-- longitudPista: integer (nullable = true)
 |-- anchoPista: integer (nullable = true)
 |-- pbmo: integer (nullable = true)
 |-- elevacion: integer (nullable = true)



# Creación Tabla Registro Aeropuertos

## Construcción key Fecha-Aeropuertos

In [17]:
#aeropuerto2 = aeropuertos.alias('aeropuerto2')

aeropuertos = aeropuertos.select(year('idFechaConstruccion').alias('ano'),
                           month('idFechaConstruccion').alias('mes'),'sigla','nombreAeropuerto','categoria','tipo','departamento','municipio','latitud','longitud',
 'propietario','longitudPista','anchoPista','pbmo','elevacion')

aeropuertos = aeropuertos.withColumn("ano",aeropuertos.ano.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("mes",aeropuertos.mes.cast(IntegerType()))
aeropuertos = aeropuertos.fillna({"ano": 2017, "mes": 7})                          


## Concatenación entre el año y mes para formar llave

In [18]:
aeropuertos = aeropuertos.select(concat_ws('-', aeropuertos.ano, aeropuertos.mes)
                                         .alias("idFechaConstruccion"),'sigla','nombreAeropuerto','categoria','tipo','departamento','municipio','latitud','longitud',
 'propietario','longitudPista','anchoPista','pbmo','elevacion')

In [19]:
#aeropuertos = aeropuertos[['sigla','idFechaConstruccion','nombreAeropuerto','categoria','tipo','departamento','municipio','latitud','longitud',
 #'propietario','longitudPista','anchoPista','pbmo','elevacion']]

## Verificación 
* Limpieza de valores null- idFechaConstruccion
* Tipo de variables

In [20]:
aeropuertos.printSchema()

root
 |-- idFechaConstruccion: string (nullable = false)
 |-- sigla: string (nullable = true)
 |-- nombreAeropuerto: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: float (nullable = true)
 |-- longitud: float (nullable = true)
 |-- propietario: string (nullable = true)
 |-- longitudPista: integer (nullable = true)
 |-- anchoPista: integer (nullable = true)
 |-- pbmo: integer (nullable = true)
 |-- elevacion: integer (nullable = true)



In [21]:
aeropuertos.filter(aeropuertos.idFechaConstruccion.isNull()).collect()

[]

In [22]:
aeropuertos.filter(aeropuertos.elevacion.isNull()).collect()

[]

### Remplazo Valores Nulos


### * idFechaVigencia

cabe resaltar que de un total **242** valores de esta columna mas del **78,5%** de los valores son nulos. Por esta razón se decir eliminar esta variable

In [23]:
aeropuertos = aeropuertos.fillna({"pbmo": 2000})
aeropuertos = aeropuertos.na.replace({'nan': 'AEROCIVIL'}, 'propietario')
#aeropuertos.groupBy('pbmo').count().orderBy('count', ascending=False).show()

In [24]:
aeropuertos.select([count(when(isnan(c), c)).alias(c) for c in aeropuertos.columns]).show()

+-------------------+-----+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+
|idFechaConstruccion|sigla|nombreAeropuerto|categoria|tipo|departamento|municipio|latitud|longitud|propietario|longitudPista|anchoPista|pbmo|elevacion|
+-------------------+-----+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+
|                  0|    0|               0|        0|   0|           0|        0|      0|       0|          0|            0|         0|   0|        0|
+-------------------+-----+----------------+---------+----+------------+---------+-------+--------+-----------+-------------+----------+----+---------+



### Borrar duplicados

In [25]:
#aeropuertos = aeropuertos[''].dropDuplicates()

In [26]:
print(f'row_number: {aeropuertos.count()} ,', f'columns_number: {len(aeropuertos.columns)}')

row_number: 242 , columns_number: 14


In [27]:
aeropuertos = aeropuertos.dropDuplicates(['sigla'])

In [28]:
aeropuertos.groupBy('sigla').count().orderBy('count', ascending=False).show()

+-----+-----+
|sigla|count|
+-----+-----+
|  9MN|    1|
|  9CL|    1|
|  7HC|    1|
|  9AB|    1|
|  9CG|    1|
|  OTU|    1|
|  9LS|    1|
|  NVA|    1|
|  7GI|    1|
|  7HM|    1|
|  RVE|    1|
|  9BB|    1|
|  9MP|    1|
|  9BI|    1|
|  9CD|    1|
|  BEC|    1|
|  ACD|    1|
|  YPP|    1|
|  PUU|    1|
|  9LR|    1|
+-----+-----+
only showing top 20 rows



In [29]:
print(f'row_number: {aeropuertos.count()} ,', f'columns_number: {len(aeropuertos.columns)}')

row_number: 212 , columns_number: 14


In [30]:
aeropuertos.toPandas().to_csv('RegistroAeropuerto.csv', sep=',', index=False)

21/11/16 00:37:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
#aeropuertos.dtypes

In [ ]:
aeropuertos["idFechaConstruccion"].isnull().sum()

In [ ]:
import pandas as pd
import datetime
aeropuertos['ano'] = pd.DatetimeIndex(aeropuertos['idFechaConstruccion']).year
aeropuertos['mes'] = pd.DatetimeIndex(aeropuertos['idFechaConstruccion']).month
aeropuertos['ano'] = aeropuertos['ano'].fillna(method="backfill")
aeropuertos['mes'] = aeropuertos['mes'].fillna(method="backfill")
aeropuertos["idFechaConstruccion"] = aeropuertos["idFechaConstruccion"].fillna(method="backfill")
aeropuertos['ano'] = aeropuertos['ano'].astype(int)
aeropuertos['mes'] = aeropuertos['mes'].astype(int)


In [ ]:
aeropuertos['ano'] = aeropuertos['ano'].astype(str)
aeropuertos['mes'] = aeropuertos['mes'].astype(str)
aeropuertos["idFechaConstruccion"] = aeropuertos[['ano','mes']].apply('-'.join, axis=1)
aeropuertos 

In [ ]:
aeropuertos = aeropuertos.drop(['ano','mes'], axis=1)
aeropuertos

In [ ]:
aeropuertos=spark.createDataFrame(aeropuertos) 
aeropuertos.printSchema()
#aeropuertos.to_csv('registroaeropuerto.csv',sep=',', index=False)

In [ ]:
aeropuertos = aeropuertos.withColumn("idFechaConstruccion",aeropuertos.idFechaConstruccion.cast(StringType()))
aeropuertos = aeropuertos.withColumn("nombreAeropuerto",aeropuertos.nombreAeropuerto.cast(StringType()))
aeropuertos = aeropuertos.withColumn("categoria",aeropuertos.categoria.cast(StringType()))
aeropuertos = aeropuertos.withColumn("tipo",aeropuertos.tipo.cast(StringType()))
aeropuertos = aeropuertos.withColumn("departamento",aeropuertos.departamento.cast(StringType()))
aeropuertos = aeropuertos.withColumn("municipio",aeropuertos.municipio.cast(StringType()))
aeropuertos = aeropuertos.withColumn("latitud",aeropuertos.latitud.cast(FloatType()))
aeropuertos = aeropuertos.withColumn("longitud",aeropuertos.longitud.cast(FloatType()))
aeropuertos = aeropuertos.withColumn("propietario",aeropuertos.propietario.cast(StringType()))
aeropuertos = aeropuertos.withColumn("longitudPista",aeropuertos.longitudPista.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("anchoPista",aeropuertos.anchoPista.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("pbmo",aeropuertos.pbmo.cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("elevacion",aeropuertos.elevacion.cast(IntegerType()))

In [ ]:
aeropuertos.printSchema()

In [ ]:
#aeropuertos['idFecha']= aeropuertos.index
#aeropuertos = aeropuertos[['sigla', 'idFecha','idFechaConstruccion', 'nombreAeropuerto', 'categoria', 'tipo',
       #'departamento', 'municipio', 'latitud', 'longitud', 'propietario',
       #'longitudPista', 'anchoPista', 'pbmo', 'elevacion']]
#aeropuertos['idFechaConstruccion'].unique()

In [ ]:
#pd.merge(aeropuertos, data, on='idFechaConstruccion')


# Vuelos

In [ ]:
vuelos = spark.read.load("vuelos.csv",format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
print(vuelos.schema)

### Selección de datos

De acuerdo al modelo, se seleccionara las variables o columnas a tener en cuenta

In [ ]:
vuelos = vuelos.selectExpr('ano', 'mes','origen','destino', 
                         'tipo_equipo as tipoEquipo','tipo_vuelo as tipoVuelo',
                         'trafico as tipoTrafico','empresa','vuelos as numeroVuelos',
                          'sillas as numeroSillas', 'carga_ofrecida as cargaOfrecida', 
                           'pasajeros as numeroPasajeros','carga_bordo as cargaAbordo'
                         )

In [ ]:
vuelos.printSchema()

In [ ]:
vuelos.show()

### Forma de los datos

In [ ]:
print(f'row_number: {vuelos.count()} ,', f'columns_number: {len(vuelos.columns)}')

### Valores Nulos

In [ ]:
vuelos.select([count(when(isnan(c), c)).alias(c) for c in vuelos.columns]).show()

### Cast Column Type

In [ ]:
vuelos.printSchema()

In [ ]:
vuelos = vuelos.withColumn("numeroVuelos",vuelos["numeroVuelos"].cast(IntegerType()))
vuelos = vuelos.withColumn("numeroSillas",vuelos["numeroSillas"].cast(IntegerType()))
vuelos = vuelos.withColumn("numeroPasajeros",vuelos["numeroPasajeros"].cast(IntegerType()))
vuelos = vuelos.withColumn("cargaOfrecida",vuelos["cargaOfrecida"].cast(IntegerType()))
vuelos = vuelos.withColumn("cargaAbordo",vuelos["cargaAbordo"].cast(IntegerType()))
vuelos = vuelos.withColumn("tipoTrafico",vuelos["tipoTrafico"].cast(StringType()))

In [ ]:
vuelos.printSchema()

In [ ]:
promedio_vuelos = vuelos.agg({'numeroVuelos': 'avg'})
promedio_vuelos.show()

In [ ]:
vuelos.groupBy('tipoTrafico').count().orderBy('count', ascending = False).show()

In [ ]:
promedio_vuelos = vuelos.agg({'numeroVuelos': 'avg'})
vuelos = vuelos.fillna({'numeroVuelos': promedio_vuelos.first()[0]})

promedio_sillas = vuelos.agg({'numeroSillas': 'avg'})
vuelos = vuelos.fillna({'numeroSillas': promedio_sillas.first()[0]})

#sillas_por_avion = vuelos[["numeroSillas","tipoEquipo"]]
#promedio_sillas = sillas_por_avion.groupby("tipoEquipo").mean()
#vuelos = vuelos.fillna({'numeroSillas': promedio_sillas.first()[0]})

promedio_pasajeros = vuelos.agg({'numeroPasajeros': 'avg'})
vuelos = vuelos.fillna({'numeroPasajeros': promedio_pasajeros.first()[0]})

vuelos = vuelos.na.replace({'nan': 'N'}, 'tipoTrafico')

vuelos.groupBy('tipoTrafico').count().orderBy('count', ascending = False).show()
vuelos[['tipoEquipo', 'numeroSillas', 'numeroPasajeros', 'numeroVuelos', 'tipoVuelo']].show()

vuelos.groupBy('tipoEquipo').count().orderBy('count', ascending=False).show()

vuelos.show()

In [ ]:
vuelos.select([count(when(isnan(c), c)).alias(c) for c in vuelos.columns]).show()
vuelos.where(col("trafico").isNull()).show()

### Borrar duplicados

In [ ]:
vuelos = vuelos.dropDuplicates()

In [ ]:
print(f'row_number: {vuelos.count()} ,', f'columns_number: {len(vuelos.columns)}')

# Creacion Tablas Modelo Multidimensional

### Trayecto Vuelo

In [ ]:
Ruta = vuelos[['origen', 'destino']]
Ruta = Ruta.select(concat(Ruta.origen, Ruta.destino).alias("idRuta"),"origen", "destino")
Ruta = Ruta.dropDuplicates()
Ruta.show(truncate = False)
#Ruta.toPandas().to_csv('TrayectoVuelo.csv', sep=',', index=False)
#Ruta.write.option('encoding', 'UTF-8').csv("trayecto.csv")

In [ ]:
Ruta.toPandas().to_csv('TrayectoVuelo.csv', sep=',', index=False)

### Fecha

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth
import pandas as pd
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)
  

Fecha1 = vuelos[['ano', 'mes']]
#Fecha1 = Fecha1.withColumn("dia", lit("01"))

Fecha2 = aeropuertos[['idFechaConstruccion']]
Fecha2 = aeropuertos.select(year('idFechaConstruccion').alias('ano'),
                           month('idFechaConstruccion').alias('mes')
                           #dayofmonth('idFechaConstruccion').alias('dia')
                           )

Fecha = unionAll([Fecha1, Fecha2])
Fecha = Fecha.select(concat_ws('-',Fecha.ano, Fecha.mes)
                                         .alias("idFecha"), 'ano','mes')

Fecha.show(truncate = False)
Fecha= Fecha.toPandas()

Fecha['ano'] = Fecha['ano'].fillna(method="backfill")
Fecha['mes'] = Fecha['mes'].fillna(method="backfill")
Fecha['ano'] = Fecha['ano'].astype(int)
Fecha['mes'] = Fecha['mes'].astype(int)
Fecha['idFecha'] = Fecha['idFecha'].astype(str)

Fecha.to_csv('fecha.csv',sep=';', index=False)
#Fecha.write.option('encoding', 'UTF-8').csv("fecha.csv")

In [ ]:
#Fecha['idFecha'] = Fecha['idFecha'].astype(str)
#fechas_unicas = Fecha['idFecha'].unique()

In [ ]:
#data = pd.DataFrame(fechas_unicas, columns = ['keyFecha'])
#data['idFecha']= data.index
#data.set_index('idFecha')
#data= data[['idFecha', 'keyFecha']]
#data.head(10)

In [ ]:
#Fecha.columns=['keyFecha','ano','mes']
#Fecha['idFecha']= Fecha.index
#Fecha = Fecha[['idFecha','keyFecha','ano','mes']]
#Fecha['keyFecha'].value_counts()

In [ ]:
#Fecha_final=pd.merge(Fecha, data, on='keyFecha')
#Fecha_final= Fecha_final.drop(['idFecha_x'], axis=1)
#Fecha_final=Fecha_final[['idFecha_y','keyFecha','ano','mes']]
#Fecha_final.columns = ['idFecha','keyFecha','ano','mes']

In [ ]:
#Fecha_final

In [ ]:
print(f'row_number: {Fecha.count()} ,', f'columns_number: {len(Fecha.columns)}')

In [ ]:
print(f'row_number: {Fecha1.count()} ,', f'columns_number: {len(Fecha1.columns)}')

In [ ]:
print(f'row_number: {Fecha2.count()} ,', f'columns_number: {len(Fecha2.columns)}')

### Registro Vuelos

In [ ]:
registro_vuelos = vuelos[['ano', 'mes', 'origen', 'destino',"tipoEquipo","tipoVuelo","tipoTrafico","empresa" ,'numeroVuelos', 'numeroPasajeros', 'numeroSillas', 'cargaOfrecida', 'cargaAbordo',]]
#registro_vuelos = registro_vuelos.withColumn("dia")
registro_vuelos = registro_vuelos.select(concat_ws('-', registro_vuelos.ano, registro_vuelos.mes)
                                         .alias("idFecha"),concat(registro_vuelos.origen, registro_vuelos.destino)
                                         .alias("idRuta"), "tipoEquipo","tipoVuelo","tipoTrafico","empresa",'numeroVuelos', 'numeroPasajeros', 'numeroSillas', 'cargaOfrecida', 'cargaAbordo')
registro_vuelos = registro_vuelos.dropDuplicates()
registro_vuelos.show(truncate = False)
#registro_vuelos.write.option('encoding', 'UTF-8').csv("registroVuelos.csv")

In [ ]:
registro_vuelos.toPandas().to_csv('RegistroVuelos.csv', sep=',', index=False)

In [ ]:
#registro_vuelos.columns = ['keyFecha','idRuta','numeroVuelos','numeroPasajeros','numeroSillas','cargaOfrecida','cargaAbordo']
#registro_vuelos['idFecha']= registro_vuelos.index
#registro_vuelos = registro_vuelos[['idFecha','keyFecha','idRuta','numeroVuelos','numeroPasajeros','numeroSillas','cargaOfrecida','cargaAbordo']]
#registro_vuelos

In [ ]:
#registro_vuelos= pd.merge(registro_vuelos, data, on='keyFecha')
#registro_vuelos

In [ ]:
#registro_vuelos = registro_vuelos.drop(['idFecha_y','idFecha_x'], axis=1)
#registro_vuelos

In [ ]:
#registro_vuelos=registro_vuelos[['idFecha','keyFecha','idRuta','numeroVuelos','numeroPasajeros','numeroSillas','cargaOfrecida','cargaAbordo']]
#registro_vuelos

In [ ]:
#registro_vuelos.dtypes

In [ ]:
aeropuertos.show()